<a href="https://colab.research.google.com/github/donghwi2022/ds-section2-project/blob/main/section2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 0. 특성 설명** 

- **targets**
    - `Outcome` - 해당 게임의 승패 여부

- **features**
    - `Game Completed Date` - 게임이 끝난 시각 → 해당 컬럼 삭제 예정
    - `Team` - 소속 팀 → 크루원 데이터만 남기고 해당 컬럼은 삭제할 예정
        - Crewmate : 크루원 
        - Imposter : 임포스터 
    - `Task Completed` - 완료한 업무 수 
    - `All Tasks Completed` - 모든 업무를 완료했는지 여부
    - `Murdered` - 임포스터에게 살해당했는지 여부                   
    - `Imposter Kills` - 임포스터로 몇 명을 죽였는지 → 해당 컬럼 삭제
    - `Game Length` - 게임 시간 
    - `Ejected` - 투표로 퇴출되었는지 여부 
    - `Sabotages Fixed` - 사보타지 고친 횟수            
    - `Time to complete all tasks` - 모든 업무 완료까지 걸린 시간   
    - `Rank Change` - 게임 승패로 랭크 변화여부 → 해당 컬럼 삭제
    - `Region/Game Code` - 게임 진행 지역/게임 코드 → 해당 컬럼 삭제 

# **Part 1. 필요 라이브러리 및 데이터 로드**

- 필요 라이브러리 import 및 한글 깨짐문제 해결
- csv 파일 load

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#한글 깨짐문제 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic');

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (3,976 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
# 파일 경로로 필요 csv 파일 불러오기
url = "/content/drive/MyDrive/codestates/files/AmongUs_User.csv"
df = pd.read_csv(url)
df.head()

,Game Completed Date,Team,Outcome,Task Completed,All Tasks Completed,Murdered,Imposter Kills,Game Length,Ejected,Sabotages Fixed,Time to complete all tasks,Rank Change,Region/Game Code
0,12/13/2020 at 1:26:56 am EST,Crewmate,Win,3,No,Yes,-,07m 04s,No,2.0,-,++,NA / WYMSBF
1,12/13/2020 at 1:17:42 am EST,Crewmate,Loss,7,Yes,No,-,16m 21s,No,1.0,09m 48s,--,NA / WYMSBF
2,12/13/2020 at 12:57:47 am EST,Crewmate,Win,3,No,No,-,11m 33s,No,0.0,-,++,NA / WYMSBF
3,12/13/2020 at 12:41:55 am EST,Imposter,Win,-,-,-,2,08m 05s,No,NaN,-,+++,Europe / QIRTNF
4,12/13/2020 at 12:30:37 am EST,Crewmate,Loss,4,No,No,-,05m 10s,No,0.0,-,---,Europe / QIRTNF


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2227 entries, 0 to 2226
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Game Completed Date         2227 non-null   object 
 1   Team                        2227 non-null   object 
 2   Outcome                     2227 non-null   object 
 3   Task Completed              2227 non-null   object 
 4   All Tasks Completed         2227 non-null   object 
 5   Murdered                    2227 non-null   object 
 6   Imposter Kills              2227 non-null   object 
 7   Game Length                 2227 non-null   object 
 8   Ejected                     2227 non-null   object 
 9   Sabotages Fixed             1761 non-null   float64
 10  Time to complete all tasks  2227 non-null   object 
 11  Rank Change                 1828 non-null   object 
 12  Region/Game Code            2227 non-null   object 
dtypes: float64(1), object(12)
memory 

# **Part 2. 데이터 전처리 및 EDA**

## **Part 2-1. 데이터 전처리 진행** 

In [56]:
# 사용하지 않는 특성 제거
df.drop(['Game Completed Date', 'Imposter Kills', 'Rank Change', 'Region/Game Code'], axis = 1, inplace = True)

# Team == 크루원에 대해서 진행할 것이기 때문에 해당 데이터만 추출후 해당 특성 제거
df_crew = df[df['Team'] == 'Crewmate'].copy()
df_crew.drop('Team', axis = 1, inplace = True)

# 데이터프레임 정보 확인
df_crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1761 entries, 0 to 2226
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Outcome                     1761 non-null   object 
 1   Task Completed              1761 non-null   object 
 2   All Tasks Completed         1761 non-null   object 
 3   Murdered                    1761 non-null   object 
 4   Game Length                 1761 non-null   object 
 5   Ejected                     1761 non-null   object 
 6   Sabotages Fixed             1761 non-null   float64
 7   Time to complete all tasks  1761 non-null   object 
dtypes: float64(1), object(7)
memory usage: 123.8+ KB


In [57]:
# 결측치 확인
df_crew.isnull().sum()

Outcome                       0
Task Completed                0
All Tasks Completed           0
Murdered                      0
Game Length                   0
Ejected                       0
Sabotages Fixed               0
Time to complete all tasks    0
dtype: int64

In [58]:
# 중복값 확인
print('중복값의 개수 : ', df_crew.duplicated().sum())

# 중복값 제거
df_crew.drop_duplicates(inplace = True)
print('중복값 제거 후, 중복값의 개수 : ', df_crew.duplicated().sum())

중복값의 개수 :  101
중복값 제거 후, 중복값의 개수 :  0


In [71]:
a = '05m 10s'
int(a.split('m')[0]) * 60 + int(a.split('m')[1][:3])

310

## **Part 2-2. EDA 진행**

# **Part 3. 머신러닝**

# **Part 4. ML모델 해석**